<a href="https://colab.research.google.com/github/kiakass/recommendation/blob/main/lightFM_jameshope_movies_blog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install lightfm

In [28]:
from lightfm import LightFM
from scipy.sparse import coo_matrix as sp
import time
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import csv
import requests
import json
from itertools import islice
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split 

import numpy as np
import pandas as pd
import warnings # warings 제거
warnings.filterwarnings(action='ignore')

In [29]:
movies_fn = ('https://raw.githubusercontent.com/kiakass/share/master/movies.csv')   # 9,125 건, 
ratings_fn = ('https://raw.githubusercontent.com/kiakass/share/master/ratings.csv') # 100,004 건
users_fn =('https://raw.githubusercontent.com/kiakass/share/master/user_features.csv')
#ratings_fn = ('C:/03.DATA/recommend_cf.csv')

In [30]:
ratings=pd.read_csv(ratings_fn).fillna(0)
ratings.drop(['timestamp'],axis=1,inplace=True)
ratings

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [31]:
movies=pd.read_csv(movies_fn).fillna(0)
movies['year'] = movies['title'].str[-5:-1]
movies

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,2016
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,2016
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,2016
9123,164977,The Gay Desperado (1936),Comedy,1936


In [32]:
pd.read_csv(movies_fn).isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

userId 에 매핑되는 movieId 로만 테이블을 만든다.

In [33]:
tmp_movie=ratings.groupby('movieId')['rating'].mean()
movies=pd.merge(tmp_movie,movies, on='movieId')
movies

,movieId,rating,title,genres,year
0,1,3.872470,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,3.401869,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,3.161017,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,2.384615,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,3.267857,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...,...
9061,161944,5.000000,The Last Brickmaker in America (2001),Drama,2001
9062,162376,4.500000,Stranger Things,Drama,hing
9063,162542,5.000000,Rustom (2016),Romance|Thriller,2016
9064,162672,3.000000,Mohenjo Daro (2016),Adventure|Drama|Romance,2016


In [34]:
users=pd.read_csv(users_fn).fillna(0)
users

,userId,gender,age
0,1,FEMAIL,30
1,2,MAIL,40
2,3,MAIL,30
3,4,MAIL,50
4,5,FEMAIL,20
...,...,...,...
666,667,MAIL,10
667,668,FEMAIL,20
668,669,MAIL,40
669,670,MAIL,60


In [35]:
from lightfm.data import Dataset
from scipy.io import mmwrite

### data 를 array 형태로 만든다

In [36]:
ratings_source = [(ratings['userId'][i], ratings['movieId'][i]) for i in range(ratings.shape[0])]

item_meta = movies[['movieId', 'title', 'genres', 'rating', 'year']]

item_features_source = [(item_meta['movieId'][i],
                        [item_meta['genres'][i],
                         item_meta['year'][i],
                         item_meta['rating'][i]]) for i in range(item_meta.shape[0])]

user_meta = users[['userId', 'gender', 'age']]

user_features_source = [(user_meta['userId'][i],
                        [user_meta['gender'][i],
                         user_meta['age'][i]]) for i in range(user_meta.shape[0])]

In [37]:
#결측치가 있는지 확인
df=pd.DataFrame(item_features_source);print(pd.DataFrame(item_features_source).head(1))
df.isnull().sum()

   0                                                  1
0  1  [Adventure|Animation|Children|Comedy|Fantasy, ...


0    0
1    0
dtype: int64

In [38]:
dataset = Dataset()
dataset.fit(users=ratings['userId'],
            items=ratings['movieId'])

interactions, weights = dataset.build_interactions(ratings_source)
dataset.fit(users=ratings['userId'],
            items=ratings['movieId'],
            item_features=item_meta[item_meta.columns[1:]].values.flatten()
            )
item_features=dataset.build_item_features(item_features_source)
dataset.fit(users=ratings['userId'],
            items=ratings['movieId'],
            user_features=user_meta[user_meta.columns[1:]].values.flatten()
            )
user_features=dataset.build_user_features(user_features_source)

#### movies 에 결측치를 제거해야한다. 즉, item_feature_source의 movieId가 userId에 100% join 되어야함

In [39]:
# Save
mmwrite('interactions.mtx', interactions)
mmwrite('item_features.mtx', item_features)
mmwrite('user_features.mtx', item_features)
mmwrite('weights.mtx', weights)

In [40]:
from lightfm.cross_validation import random_train_test_split
# Split Train, Test data
model = LightFM(loss='warp')

(train, test) = random_train_test_split(interactions=interactions, test_percentage=0.2)

model.fit(train, item_features=item_features,user_features=user_features, epochs=2)


In [41]:
train_precision = precision_at_k(model, train, k=5, item_features=item_features,user_features=user_features).mean()
test_precision = precision_at_k(model, test, k=5, item_features=item_features,user_features=user_features).mean()
train_auc = auc_score(model, train, item_features=item_features,user_features=user_features).mean()
test_auc = auc_score(model, test, item_features=item_features,user_features=user_features).mean()
# test_auc = auc_score(model, test, item_features=item_features).mean()
output = -test_precision #precision은 높을수록 좋음 : 높은 숫자를 찾기위해 음수화

print( 'Precision: train %.4f, test %.4f, AUC: train %.4f, test %.4f.' 
      % ( train_precision,test_precision,train_auc, test_auc))

Precision: train 0.3428, test 0.0787, AUC: train 0.9046, test 0.8911.


In [42]:
model.get_params(deep=True)

{'epsilon': 1e-06,
 'item_alpha': 0.0,
 'k': 5,
 'learning_rate': 0.05,
 'learning_schedule': 'adagrad',
 'loss': 'warp',
 'max_sampled': 10,
 'n': 10,
 'no_components': 10,
 'random_state': RandomState(MT19937) at 0x7FA4396EEE20,
 'rho': 0.95,
 'user_alpha': 0.0}

In [43]:
labels = np.array(movies['movieId'])

def sample_recommendation(model, data, user_ids):

    n_users, n_items = data.shape

    #build a structure to store user scores for each item
    all_scores = np.empty(shape=(0,n_items))

    #iterate through the group and build the scores
    for user_id in user_ids:
        #known_positives = labels[data.tocsr()[user_id].indices]
        #user의 전체 0~9066 movie의 예측 평점
        scores = model.predict(user_id,np.arange(n_items),item_features) 
        
        # movieId로 치환
        top_items_for_user = labels[np.argsort(-scores)] #argsort():오름차순인덱스반환
        print("Top Recommended ISBN For User: ", user_id)
        for x in top_items_for_user[:10]:
            #print("     %s" % movies[movies['movieId']== x])
            print("     %s" % x)

        #vertically stack the user scores (items are columns)
        all_scores = np.vstack((all_scores, scores))
        print(all_scores)

    #compute the average rating for each item in the group
    item_averages = np.mean(all_scores.astype(np.float), axis=0)
    top_items_for_group = labels[np.argsort(-item_averages)]
    top_items_scores=item_averages[np.argsort(-item_averages)]

    print("Top Recommended ISBN for Group:")

    for x in top_items_for_group[:10]:
        print("     %s" % x)
    return  top_items_for_group, top_items_scores

In [44]:
#################################
#                               #
#  Sampling Recommended Events  #
#                               #
#################################

#fetch user_ids of users in group
group = [1]
list_no = 20

#sample recommendations for the group
list,avg_scores = sample_recommendation(model, interactions, group)
pd.DataFrame(list).to_csv('movie.csv')
recommend_movie=pd.DataFrame(columns=movies.columns)
for x in list[:list_no]:
  recommend_movie=recommend_movie.append(movies[movies['movieId']== x], ignore_index=True)

recommend_movie['predict_score']= avg_scores[:list_no]
recommend_movie

Top Recommended ISBN For User:  1
     78
     485
     564
     52
     28
     167
     61
     62
     34
     475
[[-0.65142381 -0.77288854 -0.71841067 ... -1.21552551 -1.27825558
  -1.43717992]]
Top Recommended ISBN for Group:
     78
     485
     564
     52
     28
     167
     61
     62
     34
     475


,movieId,rating,title,genres,year,predict_score
0,78,3.200000,"Crossing Guard, The (1995)",Action|Crime|Drama|Thriller,1995,-0.181541
1,485,2.886792,Last Action Hero (1993),Action|Adventure|Comedy|Fantasy,1993,-0.198446
2,564,3.000000,Chasers (1994),Comedy,1994,-0.239077
3,52,3.637255,Mighty Aphrodite (1995),Comedy|Drama|Romance,1995,-0.245912
4,28,4.083333,Persuasion (1995),Drama|Romance,1995,-0.263395
5,167,4.000000,Feast of July (1995),Drama,1995,-0.271393
6,61,3.571429,Eye for an Eye (1996),Drama|Thriller,1996,-0.275218
7,62,3.689655,Mr. Holland's Opus (1995),Drama,1995,-0.291740
8,34,3.601351,Babe (1995),Children|Drama,1995,-0.295940
9,475,4.209677,In the Name of the Father (1993),Drama,1993,-0.305473


### Item_features

In [45]:
movies

,movieId,rating,title,genres,year
0,1,3.872470,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,3.401869,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,3.161017,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,2.384615,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,3.267857,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...,...
9061,161944,5.000000,The Last Brickmaker in America (2001),Drama,2001
9062,162376,4.500000,Stranger Things,Drama,hing
9063,162542,5.000000,Rustom (2016),Romance|Thriller,2016
9064,162672,3.000000,Mohenjo Daro (2016),Adventure|Drama|Romance,2016


In [46]:
print(item_features)  # 2열 : itemid/genre/rating embedding ex. 0,9104,9148

  (0, 0)	0.25
  (0, 9069)	0.25
  (0, 9105)	0.25
  (0, 9149)	0.25
  (1, 1)	0.25
  (1, 10413)	0.25
  (1, 10590)	0.25
  (1, 10591)	0.25
  (2, 2)	0.25
  (2, 9156)	0.25
  (2, 9210)	0.25
  (2, 9331)	0.25
  (3, 3)	0.25
  (3, 9885)	0.25
  (3, 10091)	0.25
  (3, 10714)	0.25
  (4, 4)	0.25
  (4, 9105)	0.25
  (4, 10058)	0.25
  (4, 10749)	0.25
  (5, 5)	0.25
  (5, 9167)	0.25
  (5, 10782)	0.25
  (5, 10925)	0.25
  (6, 6)	0.25
  :	:
  (9059, 19890)	0.25
  (9060, 319)	0.25
  (9060, 9060)	0.25
  (9060, 10060)	0.25
  (9060, 17761)	0.25
  (9061, 9061)	0.25
  (9061, 9552)	0.25
  (9061, 10050)	0.25
  (9061, 16879)	0.25
  (9062, 9062)	0.25
  (9062, 9223)	0.25
  (9062, 16951)	0.25
  (9062, 18106)	0.25
  (9063, 319)	0.25
  (9063, 9063)	0.25
  (9063, 9074)	0.25
  (9063, 9210)	0.25
  (9064, 417)	0.25
  (9064, 9064)	0.25
  (9064, 9077)	0.25
  (9064, 13873)	0.25
  (9065, 417)	0.25
  (9065, 9065)	0.25
  (9065, 9080)	0.25
  (9065, 10255)	0.25


In [47]:
movies.rating.value_counts() #2085 -> 962개

4.000000    962
3.000000    848
3.500000    776
2.500000    445
2.000000    377
           ... 
3.820755      1
4.183398      1
3.397959      1
3.102041      1
2.888060      1
Name: rating, Length: 1279, dtype: int64

In [48]:
movies.genres.value_counts()  # 9105 -> 1156개

Drama                                                             1156
Comedy                                                             804
Comedy|Drama                                                       424
Documentary                                                        359
Drama|Romance                                                      354
                                                                  ... 
Adventure|Musical|Romance                                            1
Action|Crime|Drama|Film-Noir|Thriller                                1
Comedy|Fantasy|Musical                                               1
Adventure|Children|Drama|Fantasy                                     1
Action|Animation|Crime|Drama|Film-Noir|Mystery|Sci-Fi|Thriller       1
Name: genres, Length: 901, dtype: int64